In [5]:
import jsonlines

transcription_path = "../first_tests/output/Debatten12okt.jsonl"
data = []
with jsonlines.open(transcription_path) as reader:
    for obj in reader:
        data.append(obj)

In [25]:
import numpy as np

def segment_sentences(start, end, txt):
    if "." not in txt:
        return [{"start": start, "end": end, "text": txt.strip()}]
    sentences = [s.strip() for s in txt.split(".") if len(s.strip()) > 0]
    n_sents = len(sentences)
    time_range = np.linspace(start, end, n_sents + 1)
    segments = []
    for i, sentence in enumerate(sentences):
        _start = np.round(time_range[i], 2)
        _end = np.round(time_range[i+1], 2)
        segments.append({"start": _start, "end": _end, "text": sentence})
    return segments

start=4
end=15
txt = "This is a sentence. This is another sentence. This is a third sentence."

for ss in segment_sentences(start, end, txt):
    print(ss)

for ss in segment_sentences(0, 3, "This is a sentence."):
    print(ss)

{'start': 4.0, 'end': 7.67, 'text': 'This is a sentence'}
{'start': 7.67, 'end': 11.33, 'text': 'This is another sentence'}
{'start': 11.33, 'end': 15.0, 'text': 'This is a third sentence'}
{'start': 0.0, 'end': 3.0, 'text': 'This is a sentence'}


In [34]:
import numpy as np

parsed = []
for obj in data:
    timestamp = obj["timestamp"]
    txt = obj["text"]
    start, end = timestamp
    parsed.extend(segment_sentences(start, end, txt))

In [41]:
import pandas as pd
df = pd.DataFrame(parsed)
df.head()

,start,end,text
0,3.96,5.62,Hei
1,5.62,7.28,Denne debatten skal handle om dette
2,7.28,11.50,Målet er at flest mulig skal jobbe
3,11.50,15.72,Derfor må det lønne seg å stå opp om morgenen
4,15.76,17.52,Og dette


In [37]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("NbAiLab/nb-sbert-base")

In [43]:
embeddings = model.encode(df["text"].tolist(), show_progress_bar=True);

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

In [44]:
from autofaiss import build_index
index, index_infos = build_index(embeddings, save_on_disk=False);

2023-10-27 13:02:06,448 [INFO]: Using 10 omp threads (processes), consider increasing --nb_cores if you have more
2023-10-27 13:02:06,450 [INFO]: Launching the whole pipeline 10/27/2023, 13:02:06
2023-10-27 13:02:06,450 [INFO]: Reading total number of vectors and dimension 10/27/2023, 13:02:06
100%|██████████| 1/1 [00:00<00:00, 33554.43it/s]
2023-10-27 13:02:06,459 [INFO]: There are 687 embeddings of dim 768
2023-10-27 13:02:06,459 [INFO]: >>> Finished "Reading total number of vectors and dimension" in 0.0091 secs
2023-10-27 13:02:06,460 [INFO]: 	Compute estimated construction time of the index 10/27/2023, 13:02:06
2023-10-27 13:02:06,460 [INFO]: 		-> Train: 16.7 minutes
2023-10-27 13:02:06,460 [INFO]: 		-> Add: 0.0 seconds
2023-10-27 13:02:06,461 [INFO]: 		Total: 16.7 minutes
2023-10-27 13:02:06,461 [INFO]: 	>>> Finished "Compute estimated construction time of the index" in 0.0013 secs
2023-10-27 13:02:06,461 [INFO]: 	Checking that your have enough memory available to create the index

In [51]:
def query(q, K=10):
    emb = model.encode([q], show_progress_bar=False)
    _, matches = index.search(emb, K)
    return df.iloc[matches[0]]

query("trygd og skatt")

,start,end,text
384,1504.00,1505.04,Det er en inntektssikkerhet som gjør at man ka...
631,2412.20,2414.36,"Ja, inntektsgrense og kompensasjonsgrad"
78,335.08,339.72,Det er et spleiselag hvor de som jobber hver d...
386,1507.96,1510.80,Det er en inntektssikring
308,1191.00,1192.98,Det er staten som blir kompensert
48,218.04,220.78,Så legger vi sammen denne reduserte trygden
105,424.60,425.32,"Det er en utfordring for den enkelte,ge uføre"
28,123.84,126.12,Det gir svaret på hvor mye Lars sin trygd
305,1181.64,1184.00,"Kuttet, som vi kaller kompensasjonsgrad"
377,1471.04,1476.68,"Det har noe med størrelsen på ytelsen å gjøre,..."
